In [2]:
# from NLPutils.models.dilatedconv import DilatedConv
from NLPutils.models.dconv2 import DConv

In [3]:
# from NLPutils.trainers.dconv_trainer import DConvTrainer
from NLPutils.trainers.dconv2 import DConvTrainer

/src/NLPutils/trainers/dconv2.py:55: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(length == len(guess[b]), "lengths differ: length-- {}, len(guess[b])-- {} ".format(length, len(guess[b])))


In [4]:
train = '/data/ner/conll/eng.train'
valid = '/data/ner/conll/eng.testa'
test = '/data/ner/conll/eng.testb'
word_embed_loc = '/data/embeddings/GoogleNews-vectors-negative300.bin'
valsplit=0.15

In [5]:
from NLPutils.util.conll_util import *
from NLPutils.embeddings import Word2VecModel, RandomInitVecModel

maxs, maxw, vocab_ch, vocab_word = conllBuildVocab([train, valid,
                                                    test])

# Vocab LUTs
word_vocab = None
char_vocab = None

word_vec = Word2VecModel(word_embed_loc, vocab_word, 0.25)
word_vocab = word_vec.vocab

# if FLAGS.charsz != FLAGS.wsz and FLAGS.cbow is True:
#     print('Warning, you have opted for CBOW char embeddings, but have provided differing sizes for char embedding depth and word depth.  This is not possible, forcing char embedding depth to be word depth ' + FLAGS.wsz)
#     FLAGS.charsz = FLAGS.wsz

char_vec = RandomInitVecModel(16, vocab_ch, 0.25)
char_vocab = char_vec.vocab

f2i = {"<PAD>":0}

ts, f2i, _ = conllSentsToIndices(train, word_vocab, char_vocab, maxs, maxw, f2i, 3)
print(len(ts))
print('Loaded  training data')

if valid is not None:
    print('Using provided validation data')
    vs, f2i,_ = conllSentsToIndices(valid, word_vocab, char_vocab, maxs, maxw, f2i, 3)
else:
    ts, vs = validSplit(ts, valsplit)
    print('Created validation split')


es, f2i,txts = conllSentsToIndices(test, word_vocab, char_vocab, maxs, maxw, f2i, 3)
print('Loaded test data')


14986
Loaded  training data
Using provided validation data
Loaded test data


In [6]:
print('word embed weights shape: ', word_vec.weights.shape)
print( 'max sentence len', maxs)

word embed weights shape:  (30291, 300)
max sentence len 124


In [7]:
import tensorflow as tf

trainer = DConvTrainer(None, DConv, 'conll-ner-dconv-7')
trainer.train('ner', ts, f2i, vs, es,
             char_vec,
             word_vec,
             'eval',
              batchsz=2**5,
              optim='adam',
              eta=0.0005,
              epochs=500,
              dropout=0.15,
              patience=50,
              cfiltsz='1,3,5,7',
             maxlen=maxs,
             maxw=maxw,
              num_filt=300,
             num_layers=3,
             num_iterations=2,
             fscore=1,
              viz=1,
             crf=True)

#1 -- 86.55 dropout at .45 and 300 filters. stopped at epoc 24.
#2 -- 86.7 dropout at .35, 350 filters.
#3 -- added dilation 1 conv at end of block. 86.28
#4 -- 3 layers, 3 iters. got much worse. peeked at 91.8 in training, started dropping.
#5 -- layers in block should be 1,2,1 from strubell's code. lowered dropout to .15 -- layers would have been 1,1,1.
#6 -- clipped gradients'
#7 -- 3 layers, 2 iterations.

crf=True, creating SLL
crf=True, creating SLL


/opt/conda/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Setting up word embedding visualization
Writing metadata
Training epoch 1.
Train (Loss 0.1324) (169.546 sec)
Validation (F1 = 0.7211) (Acc 49069/51409 = 0.9545) (8.956 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 2.
	(last improvement @ 1)
Train (Loss 0.0423) (161.183 sec)
Validation (F1 = 0.8777) (Acc 50338/51409 = 0.9792) (8.697 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 3.
	(last improvement @ 2)
Train (Loss 0.0238) (160.942 sec)
Validation (F1 = 0.8911) (Acc 50471/51409 = 0.9818) (8.799 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 4.
	(last improvement @ 3)
Train (Loss 0.0159) (161.319 sec)
Validation (F1 = 0.9083) (Acc 50601/51409 = 0.9843) (8.820 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 5.
	(last improvement @ 4)
Train (Loss 0.0116) (160.972 sec)
Validation (F1 = 0.9123) (Acc 50657/51409 = 0.9854) (8.738 sec)
Highest dev F1 achieved yet -- writing model
Training epoch 6.
	(last improvement @ 5)


KeyboardInterrupt: 